<a href="https://colab.research.google.com/github/muhozag/sparklearning/blob/nlp/nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Natural Language Processing Example: Detecting Spam**

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('nlp').getOrCreate()

In [2]:
data = spark.read.csv("SMSSpamCollection",inferSchema=True,sep='\t')

In [3]:
data = data.withColumnRenamed('_c0','class').withColumnRenamed('_c1','text')

In [4]:
data.show()

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
| spam|FreeMsg Hey there...|
|  ham|Even my brother i...|
|  ham|As per your reque...|
| spam|WINNER!! As a val...|
| spam|Had your mobile 1...|
|  ham|I'm gonna be home...|
| spam|SIX chances to wi...|
| spam|URGENT! You have ...|
|  ham|I've been searchi...|
|  ham|I HAVE A DATE ON ...|
| spam|XXXMobileMovieClu...|
|  ham|Oh k...i'm watchi...|
|  ham|Eh u remember how...|
|  ham|Fine if thats th...|
| spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows



In [5]:
from pyspark.sql.functions import length

In [6]:
data = data.withColumn('length',length(data['text']))

In [7]:
data.show()

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
| spam|FreeMsg Hey there...|   147|
|  ham|Even my brother i...|    77|
|  ham|As per your reque...|   160|
| spam|WINNER!! As a val...|   157|
| spam|Had your mobile 1...|   154|
|  ham|I'm gonna be home...|   109|
| spam|SIX chances to wi...|   136|
| spam|URGENT! You have ...|   155|
|  ham|I've been searchi...|   196|
|  ham|I HAVE A DATE ON ...|    35|
| spam|XXXMobileMovieClu...|   149|
|  ham|Oh k...i'm watchi...|    26|
|  ham|Eh u remember how...|    81|
|  ham|Fine if thats th...|    56|
| spam|England v Macedon...|   155|
+-----+--------------------+------+
only showing top 20 rows



In [8]:
data.groupBy('class').mean().show()

+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



In [9]:
from pyspark.ml.feature import Tokenizer,StopWordsRemover,CountVectorizer,IDF,StringIndexer
tokenizer = Tokenizer(inputCol="text", outputCol="token_text")

In [10]:
tokenizer = Tokenizer(inputCol="text", outputCol="token_text")

In [22]:
stop_remove = StopWordsRemover(inputCol='token_text',outputCol='stop_tokens')

In [23]:
count_vec = CountVectorizer(inputCol='stop_tokens',outputCol='c_vec')

In [12]:
idf = IDF(inputCol="c_vec", outputCol="tf_idf")

In [14]:
ham_spam_to_num = StringIndexer(inputCol='class',outputCol='label')

In [15]:
from pyspark.ml.feature import VectorAssembler

In [16]:
clean_up = VectorAssembler(inputCols=['tf_idf','length'],outputCol='features')

In [17]:
from pyspark.ml.classification import NaiveBayes

In [19]:
nb = NaiveBayes()

In [20]:
from pyspark.ml import Pipeline

In [24]:
data_prep_pipe = Pipeline(stages=[ham_spam_to_num,tokenizer,
                                  stop_remove,count_vec,idf,clean_up])

In [25]:
final_data = data_prep_pipe.fit(data).transform(data)

In [26]:
final_data.show()

+-----+--------------------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
|class|                text|length|label|          token_text|         stop_tokens|               c_vec|              tf_idf|            features|
+-----+--------------------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
|  ham|Go until jurong p...|   111|  0.0|[go, until, juron...|[go, jurong, poin...|(13423,[7,11,31,6...|(13423,[7,11,31,6...|(13424,[7,11,31,6...|
|  ham|Ok lar... Joking ...|    29|  0.0|[ok, lar..., joki...|[ok, lar..., joki...|(13423,[0,24,301,...|(13423,[0,24,301,...|(13424,[0,24,301,...|
| spam|Free entry in 2 a...|   155|  1.0|[free, entry, in,...|[free, entry, 2, ...|(13423,[2,13,19,3...|(13423,[2,13,19,3...|(13424,[2,13,19,3...|
|  ham|U dun say so earl...|    49|  0.0|[u, dun, say, so,...|[u, dun, say, ear...|(13423,[0,70,80,1...|(13423,[0,70,8

In [27]:
clean_data = final_data.select(['label','features'])

In [28]:
clean_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(13424,[7,11,31,6...|
|  0.0|(13424,[0,24,301,...|
|  1.0|(13424,[2,13,19,3...|
|  0.0|(13424,[0,70,80,1...|
|  0.0|(13424,[36,134,31...|
|  1.0|(13424,[10,60,140...|
|  0.0|(13424,[10,53,102...|
|  0.0|(13424,[127,185,4...|
|  1.0|(13424,[1,47,121,...|
|  1.0|(13424,[0,1,13,27...|
|  0.0|(13424,[18,43,117...|
|  1.0|(13424,[8,16,37,8...|
|  1.0|(13424,[13,30,47,...|
|  0.0|(13424,[39,95,221...|
|  0.0|(13424,[555,1797,...|
|  1.0|(13424,[30,109,11...|
|  0.0|(13424,[82,214,44...|
|  0.0|(13424,[0,2,49,13...|
|  0.0|(13424,[0,74,105,...|
|  1.0|(13424,[4,30,33,5...|
+-----+--------------------+
only showing top 20 rows



In [29]:
training_data,test_data = clean_data.randomSplit([0.7,0.3])

In [30]:
spam_detector = nb.fit(training_data)

In [31]:
test_results = spam_detector.transform(test_data)

In [32]:
test_results.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(13424,[0,1,2,13,...|[-610.09332309054...|[1.0,2.8165051964...|       0.0|
|  0.0|(13424,[0,1,7,8,1...|[-873.17629376534...|[1.0,2.7798794983...|       0.0|
|  0.0|(13424,[0,1,9,14,...|[-565.90850503327...|[1.0,1.2659787979...|       0.0|
|  0.0|(13424,[0,1,9,14,...|[-565.90850503327...|[1.0,1.2659787979...|       0.0|
|  0.0|(13424,[0,1,11,32...|[-870.47536543124...|[1.0,3.8766500402...|       0.0|
|  0.0|(13424,[0,1,15,20...|[-672.83077910620...|[1.0,3.8949271323...|       0.0|
|  0.0|(13424,[0,1,18,20...|[-853.35294915758...|[1.0,5.1527123543...|       0.0|
|  0.0|(13424,[0,1,71,10...|[-684.47659596150...|[1.0,1.6440114860...|       0.0|
|  0.0|(13424,[0,2,3,6,9...|[-3286.9782847243...|[1.0,4.9135849417...|       0.0|
|  0.0|(13424,[0

In [33]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [34]:
acc_evall = MulticlassClassificationEvaluator()

In [35]:
acc = acc_evall.evaluate(test_results)

In [36]:
acc

0.931108963343087